In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime  
import time
from hashlib import sha1
import hmac
from wsgiref.handlers import format_date_time
from datetime import datetime
from time import mktime
import base64

In [2]:
app_id = '2d16769d481b4ab69ec6ecb0ccd0672b'
app_key = 'dbr3zEbzwX8cj1Yv7ZxGzb4IpAc'
class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        xdate = format_date_time(mktime(datetime.now().timetuple()))
        hashed = hmac.new(self.app_key.encode('utf8'), ('x-date: ' + xdate).encode('utf8'), sha1)
        signature = base64.b64encode(hashed.digest()).decode()

        authorization = 'hmac username="' + self.app_id + '", ' + \
                        'algorithm="hmac-sha1", ' + \
                        'headers="x-date", ' + \
                        'signature="' + signature + '"'
        return {
            'Authorization': authorization,
            'x-date': format_date_time(mktime(datetime.now().timetuple())),
            'Accept - Encoding': 'gzip'
        }

In [7]:
a = Auth(app_id, app_key)
r=requests.get("https://ptx.transportdata.tw/MOTC/v2/Bus/StopOfRoute/InterCity?$format=xml", headers= a.get_auth_header())
r.close()
soup = BeautifulSoup(r.text, "html.parser")
data_all=pd.DataFrame(columns = ["路線唯一識別代碼","地區既用中之路線代碼","路線中文名稱","路線英文名稱","營運業者代碼"
                                 ,"營運業者中文名稱","營運業者英文名稱","營運業者簡碼","營運業者編號","附屬路線唯一識別代碼"
                                 ,'附屬路線代碼',"附屬路線中文名稱",'附屬路線英文名稱',"去返程","資料更新日期時間","資料版本編號"
                                 ,"經此站牌唯一識別代碼","經此站牌代碼","經此站牌中文名稱","經此站牌英文名稱"
                                 ,"經此站牌上下車站別","路線經過站牌之順序","經此站牌位置緯度","經此站牌位置經度","經此站牌地理空間編碼"
                                 ,"經此站牌所屬的站位ID","站牌所屬的組站位ID","經此站牌位置縣市之代碼"])
for i in soup.find_all('busstopofroute'): #每次跑一個站的資料
    #print(i.prettify())
    #print(j)
    #for j in i.find_all("stops"):
    routeuid = [routeuid.text for routeuid in i.find_all('routeuid')]
    routeid = [routeid.text for routeid in i.find_all('routeid')]
    route_zh_tw = [i.zh_tw.text for i in i.find_all('routename')]
    route_en = [i.en.text for en in i.find_all('routename')]
    operatorid = [operatorid.text for operatorid in i.find_all('operatorid')]
    operatorid_zh_tw = [ i.zh_tw.text for i in i.find_all('operatorname')]
    try:
        operatorid_en = [ i.en.text for i in i.find_all('operatorname')]
    except AttributeError:
        operatorid_en=[]
    operatorcode = [operatorcode.text for operatorcode in i.find_all('operatorcode')]
    operatorno = [operatorno.text for operatorno in i.find_all('operatorno')]
    subrouteuid = [subrouteuid.text for subrouteuid in i.find_all('subrouteuid')]
    subrouteid = [subrouteid.text for subrouteid in i.find_all('subrouteid')]
    subroutename_zh_tw = [i.zh_tw.text for zh_tw in i.find_all('subroutename')]
    subroutename_en = [i.en.text for en in i.find_all('subroutename')]
    direction = [direction.text for direction in i.find_all('direction')]
    #city = [city.text for city in i.find_all('city')]
    #citycode = [i.citycode.text for citycode in i.find_all('citycode')]
    updateTime = [i.updatetime.text for updatetime in i.find_all('updatetime')]
    versionID = [versionid.text for versionid in i.find_all('versionid')]
    print(routeuid)

    busstop_of_route={"路線唯一識別代碼":routeuid,"地區既用中之路線代碼":routeid,"路線中文名稱":route_zh_tw,"路線英文名稱":route_en
        ,"營運業者代碼":operatorid,"營運業者中文名稱":operatorid_zh_tw,"營運業者英文名稱":operatorid_en
        ,"營運業者簡碼":operatorcode,"營運業者編號":operatorno,"附屬路線唯一識別代碼":subrouteuid,'附屬路線代碼':subrouteid,
         '附屬路線中文名稱':subroutename_zh_tw,'附屬路線英文名稱':subroutename_en,"去返程":direction,"資料更新日期時間":updateTime
        ,"資料版本編號":versionID}
    #print(Bus_information)
    busstop_of_route = pd.DataFrame.from_dict(busstop_of_route, orient='index')
    busstop_of_route=busstop_of_route.transpose()#把欄位顛倒過來
    #print(busstop_of_route)
    #合併經過公車
    
    routeuid = [routeuid.text for routeuid in i.find_all('routeuid')]
    stopuid = [i.stopuid.text for i in i.find_all('stop')]
    stopid = [i.stopid.text for i in i.find_all('stop')]
    stopname_zh_tw=[i.zh_tw.text for i in i.find_all('stopname')]
    stopname_en=[i.en.text for i in i.find_all('stopname')]
    stopboarding = [i.text for i in i.find_all('stopboarding')]
    stopsequence = [i.text for i in i.find_all('stopsequence')]
    positionlat = [i.text for i in i.find_all('positionlat')]
    positionlon=[i.text for i in i.find_all('positionlon')]
    geohash = [i.text for i in i.find_all('geohash')]
    stationid=[i.text for i in i.find_all('stationid')]
    stationgroupid=[i.text for i in i.find_all('stationgroupid')]
    locationcitycode=[i.text for i in i.find_all('locationcitycode')]
    route_connection={'路線唯一識別代碼':routeuid,"經此站牌唯一識別代碼":stopuid,"經此站牌代碼":stopid,
                      "經此站牌中文名稱":stopname_zh_tw,"經此站牌英文名稱":stopname_en,"經此站牌上下車站別":stopboarding,
                      "路線經過站牌之順序":stopsequence,"經此站牌位置緯度":positionlat,"經此站牌位置經度":positionlon,
                      "經此站牌地理空間編碼":geohash,"經此站牌所屬的站位ID":stationid,"站牌所屬的組站位ID":stationgroupid,
                      "經此站牌位置縣市之代碼":locationcitycode}
    df = pd.DataFrame.from_dict(route_connection, orient='index')
    df=df.transpose()#把欄位顛倒過來
    df=df.fillna(method='pad')#用前一個值彌補後一個值
    _all=pd.merge(busstop_of_route, df,on="路線唯一識別代碼")#完整資料
    
    #Route_information=pd.concat([Route_information, _all])
    data_all=data_all.append(_all)
    time.sleep(0.5)

['THB0701']
['THB0701']
['THB0968']
['THB0968']
['THB0968']
['THB0968']
['THB0968']
['THB0968']
['THB0968']
['THB0968']
['THB1011']
['THB1011']
['THB1031']
['THB1031']
['THB1032']
['THB1032']
['THB1051']
['THB1051']
['THB1061']
['THB1061']
['THB1061']
['THB1061']
['THB1062']
['THB1062']
['THB1068']
['THB1068']
['THB1070']
['THB1070']
['THB1070']
['THB1070']
['THB1071']
['THB1071']
['THB1071']
['THB1088']
['THB1088']
['THB1120']
['THB1120']
['THB1121']
['THB1121']
['THB1122']
['THB1122']
['THB1123']
['THB1125']
['THB1125']
['THB1126']
['THB1126']
['THB1128']
['THB1128']
['THB1129']
['THB1129']
['THB1130']
['THB1130']
['THB1131']
['THB1131']
['THB1131']
['THB1131']
['THB1132']
['THB1132']
['THB1132']
['THB1132']
['THB1133']
['THB1133']
['THB1135']
['THB1135']
['THB1135']
['THB1135']
['THB1136']
['THB1136']
['THB1137']
['THB1137']
['THB1139']
['THB1139']
['THB1139']
['THB1139']
['THB1140']
['THB1140']
['THB1140']
['THB1140']
['THB1141']
['THB1141']
['THB1142']
['THB1142']
['THB1143']
['TH

['THB5634']
['THB5634']
['THB5635']
['THB5635']
['THB5636']
['THB5636']
['THB5637']
['THB5637']
['THB5638']
['THB5638']
['THB5639']
['THB5639']
['THB5640']
['THB5640']
['THB5641']
['THB5641']
['THB5642']
['THB5642']
['THB5643']
['THB5643']
['THB5655']
['THB5655']
['THB5656']
['THB5656']
['THB5656']
['THB5656']
['THB5656']
['THB5657']
['THB5657']
['THB5658']
['THB5658']
['THB5659']
['THB5659']
['THB5660']
['THB5660']
['THB5661']
['THB5661']
['THB5662']
['THB5662']
['THB5662']
['THB5662']
['THB5663']
['THB5663']
['THB5664']
['THB5664']
['THB5664']
['THB5664']
['THB5664']
['THB5664']
['THB5664']
['THB5664']
['THB5667']
['THB5667']
['THB5668']
['THB5668']
['THB5668']
['THB5668']
['THB5669']
['THB5669']
['THB5670']
['THB5670']
['THB5673']
['THB5673']
['THB5673']
['THB5673']
['THB5673']
['THB5673']
['THB5676']
['THB5676']
['THB5700']
['THB5700']
['THB5801']
['THB5801']
['THB5801']
['THB5801']
['THB5802']
['THB5802']
['THB5803']
['THB5803']
['THB5804']
['THB5804']
['THB5805']
['THB5805']
['TH

['THB7215']
['THB7215']
['THB7215']
['THB7215']
['THB7216']
['THB7216']
['THB7216']
['THB7216']
['THB7217']
['THB7217']
['THB7217']
['THB7217']
['THB7218']
['THB7218']
['THB7219']
['THB7219']
['THB7220']
['THB7220']
['THB7221']
['THB7221']
['THB7221']
['THB7221']
['THB7222']
['THB7222']
['THB7223']
['THB7223']
['THB7223']
['THB7223']
['THB7224']
['THB7224']
['THB7225']
['THB7225']
['THB7226']
['THB7226']
['THB7227']
['THB7227']
['THB7228']
['THB7228']
['THB7229']
['THB7229']
['THB7231']
['THB7231']
['THB7231']
['THB7231']
['THB7233']
['THB7233']
['THB7233']
['THB7233']
['THB7234']
['THB7234']
['THB7235']
['THB7235']
['THB7301']
['THB7301']
['THB7301']
['THB7301']
['THB7302']
['THB7302']
['THB7303']
['THB7303']
['THB7304']
['THB7304']
['THB7304']
['THB7304']
['THB7304']
['THB7304']
['THB7305']
['THB7305']
['THB7305']
['THB7306']
['THB7306']
['THB7307']
['THB7307']
['THB7307']
['THB7307']
['THB7308']
['THB7308']
['THB7309']
['THB7309']
['THB7311']
['THB7311']
['THB7311']
['THB7312']
['TH

In [20]:
data_all.to_csv(r'C:\Users\公路客運之路線與站牌資料.csv', index=False,encoding="utf_8_sig" )

In [6]:
data_all

,路線唯一識別代碼,地區既用中之路線代碼,路線中文名稱,路線英文名稱,營運業者代碼,營運業者中文名稱,營運業者英文名稱,營運業者簡碼,營運業者編號,附屬路線唯一識別代碼,...,經此站牌中文名稱,經此站牌英文名稱,經此站牌上下車站別,路線經過站牌之順序,經此站牌位置緯度,經此站牌位置經度,經此站牌地理空間編碼,經此站牌所屬的站位ID,站牌所屬的組站位ID,經此站牌位置縣市之代碼
0,THB0701,0701,0701,0701,21,臺西客運,"Taisi Bus Co., Ltd.",TaisiBus,1408,THB070101,...,臺西,Taixi,1,1,23.69884,120.19872,wsjqz4np9,103497,636－025,YUN
1,THB0701,0701,0701,0701,21,臺西客運,"Taisi Bus Co., Ltd.",TaisiBus,1408,THB070101,...,林厝寮,Lincuoliao,0,2,23.67279,120.1779,wsjqwtqzu,115103,654－035,YUN
2,THB0701,0701,0701,0701,21,臺西客運,"Taisi Bus Co., Ltd.",TaisiBus,1408,THB070101,...,三條崙,Santiaolun,0,3,23.65318,120.15699,wsjqw1ztu,113624,654－016,YUN
3,THB0701,0701,0701,0701,21,臺西客運,"Taisi Bus Co., Ltd.",TaisiBus,1408,THB070101,...,飛沙國小,Fei Sha Elementary School,0,4,23.642565,120.177227,wsjqqxyru,138419,654－011,YUN
4,THB0701,0701,0701,0701,21,臺西客運,"Taisi Bus Co., Ltd.",TaisiBus,1408,THB070101,...,東羊稠,Dongyangchou,0,5,23.638616,120.206397,wsjqrr78u,138421,654－023,YUN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,THB1133,1133,1133,1133,46,花蓮客運,"Hualien Bus Co., Ltd.",HualienBus,0805,THB113302,...,大同市場,Datong Market,0,59,23.9811100013681,121.606259954273,wsq2rjwup,105898,970－009,HUA
59,THB1133,1133,1133,1133,46,花蓮客運,"Hualien Bus Co., Ltd.",HualienBus,0805,THB113302,...,郵政總局,"Chunghwa Post Co., Ltd.",0,60,23.9840900013491,121.604059954323,wsq2rnjm7,32402,970－006,HUA
60,THB1133,1133,1133,1133,46,花蓮客運,"Hualien Bus Co., Ltd.",HualienBus,0805,THB113302,...,花蓮商校,National Hualien Commerical High School,0,61,23.9866700013317,121.601939954371,wsq2rneu6,114633,970－002,HUA
61,THB1133,1133,1133,1133,46,花蓮客運,"Hualien Bus Co., Ltd.",HualienBus,0805,THB113302,...,市立圖書館,City Library,0,62,23.99003,121.60153,wsq2rp781,128328,970－043,HUA
